In [ ]:
import numpy as np
import rebound

# Start REBOUND simulation
sim = rebound.Simulation()

# Use the IAS15 integrator, suitable for long-term integrations and close encounters
sim.integrator = 'ias15'

# Set units for the simulation (years, astronomical units, solar masses)
sim.units = ('yr', 'AU', 'Msun')

# Earth Mass in Solar Units
m_earth = 0.0000030034146856628

# Create a central star with a mass of 1.2 solar masses
sim.add(hash="Star", m=1.2) 

# Create planet 1 with a mass of 1.1 Earth masses and a semi-major axis of 1.2 AU
sim.add(hash="Planet 1", m=1.1*m_earth, a=1.2)

# Create planet 2 with a period 0.52 times that of planet 1, 
# calculated using Kepler's Third Law
a_2 = ((sim.G * 1.2 * (0.52*sim.particles[1].P)**2) / (4 * np.pi**2))**(1/3) 
sim.add(hash="Planet 2", m=1.6*m_earth, a=a_2)

# Create planet 3 with a period 0.255 times that of planet 1,
# calculated using Kepler's Third Law
a_3 = ((sim.G * 1.2 * (0.255*sim.particles[1].P)**2) / (4 * np.pi**2))**(1/3)
sim.add(hash="Planet 3", m=2.3*m_earth, a=a_3)

# Dictionary to store the measured quantities for each planet
quantities = {'Planet 1 Semi-Major Axis': [], 'Planet 1 Period': [], 'Planet 1 Eccentricity': [],
              'Planet 2 Semi-Major Axis': [], 'Planet 2 Period': [], 'Planet 2 Eccentricity': [],
              'Planet 3 Semi-Major Axis': [], 'Planet 3 Period': [], 'Planet 3 Eccentricity': []}

# List to store the time steps
time = []


# Integrate the simulation for 10 times the period of planet 3
while sim.t <= 10*sim.particles[3].P:
    sim.integrate(sim.t + sim.dt, exact_finish_time=False)  # Integrate with a small time step (sim.dt)
    
    # Calculate and store orbital parameters for each planet
    for i in range(3):
        orbit = sim.particles[i + 1].orbit(primary=sim.particles[0])  # Calculate orbit relative to the star
        quantities[f'Planet {i+1} Semi-Major Axis'].append(orbit.a)
        quantities[f'Planet {i+1} Period'].append(sim.particles[i+1].P)
        quantities[f'Planet {i+1} Eccentricity'].append(orbit.e)

    # Record current simulation time
    time.append(sim.t)

    # Move the system to the center of mass every 100 steps to improve accuracy
    if len(time) % 100 == 0:
        sim.move_to_com()

import csv

# Save the time data to a CSV file
with open('Orbital_Resonance_Data(time).csv', 'w', newline='') as csvfile:
    w = csv.writer(csvfile)
    w.writerow(time)

# Save the orbital parameters data to a separate CSV file
with open("Orbital_Resonance_Data(quantities).csv", "w", newline="") as f:
    w = csv.DictWriter(f, quantities.keys())
    w.writeheader()
    w.writerow(quantities)
